In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sf
from 

In [2]:
spark = SparkSession.builder.getOrCreate()

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_competition = spark.read.option("multiline", "true").json("s3://sor/football/data/competitions.json")

23/12/25 20:50:20 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
df_competition.printSchema

<bound method DataFrame.printSchema of DataFrame[competition_gender: string, competition_id: bigint, competition_international: boolean, competition_name: string, competition_youth: boolean, country_name: string, match_available: string, match_available_360: string, match_updated: string, match_updated_360: string, season_id: bigint, season_name: string]>

In [5]:
display(df_competition)

DataFrame[competition_gender: string, competition_id: bigint, competition_international: boolean, competition_name: string, competition_youth: boolean, country_name: string, match_available: string, match_available_360: string, match_updated: string, match_updated_360: string, season_id: bigint, season_name: string]

In [6]:
df_competition.show()

+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|competition_gender|competition_id|competition_international|competition_name|competition_youth|country_name|     match_available|match_available_360|       match_updated|   match_updated_360|season_id|season_name|
+------------------+--------------+-------------------------+----------------+-----------------+------------+--------------------+-------------------+--------------------+--------------------+---------+-----------+
|              male|             9|                    false|   1. Bundesliga|            false|     Germany|2023-12-12T07:43:...|               null|2023-12-12T07:43:...|                null|       27|  2015/2016|
|              male|            16|                    false|Champions League|            false|      Europe|2023-03-07T12:20:...|          

# Explorando os dados 

## Quais competições tem na base ?

In [7]:
df_competition.select("competition_name", "competition_id").distinct().orderBy("competition_name").show(truncate=False)

+-----------------------+--------------+
|competition_name       |competition_id|
+-----------------------+--------------+
|1. Bundesliga          |9             |
|Champions League       |16            |
|Copa del Rey           |87            |
|FA Women's Super League|37            |
|FIFA U20 World Cup     |1470          |
|FIFA World Cup         |43            |
|Indian Super league    |1238          |
|La Liga                |11            |
|Liga Profesional       |81            |
|Ligue 1                |7             |
|Major League Soccer    |44            |
|NWSL                   |49            |
|North American League  |116           |
|Premier League         |2             |
|Serie A                |12            |
|UEFA Euro              |55            |
|UEFA Europa League     |35            |
|UEFA Women's Euro      |53            |
|Women's World Cup      |72            |
+-----------------------+--------------+



## Qual é a edição de Champions League mais recente ?

In [8]:
df_champions = df_competition.select("competition_id", "competition_name", "season_id", "season_name").where("competition_id = 16").orderBy(df_competition.season_name.desc())
df_champions.show()

+--------------+----------------+---------+-----------+
|competition_id|competition_name|season_id|season_name|
+--------------+----------------+---------+-----------+
|            16|Champions League|        4|  2018/2019|
|            16|Champions League|        1|  2017/2018|
|            16|Champions League|        2|  2016/2017|
|            16|Champions League|       27|  2015/2016|
|            16|Champions League|       26|  2014/2015|
|            16|Champions League|       25|  2013/2014|
|            16|Champions League|       24|  2012/2013|
|            16|Champions League|       23|  2011/2012|
|            16|Champions League|       22|  2010/2011|
|            16|Champions League|       21|  2009/2010|
|            16|Champions League|       41|  2008/2009|
|            16|Champions League|       39|  2006/2007|
|            16|Champions League|       37|  2004/2005|
|            16|Champions League|       44|  2003/2004|
|            16|Champions League|       76|  199

In [9]:
champions_mais_recente = df_champions.agg({"season_name":"max"}).withColumnRenamed("max(season_name)", "season_mais_recente")

champions_mais_recente.show()


+-------------------+
|season_mais_recente|
+-------------------+
|          2018/2019|
+-------------------+



## Qual o time campeão da última edição da Champions League ?

In [11]:
competition_id  = 16
season_id = 4

df_jogos_ultima_temporada_champions = spark.read.option("multiline", "true").json(f"s3://sor/football/data/matches/{competition_id}/{season_id}.json")



In [13]:
df_jogos_ultima_temporada_champions.printSchema()

root
 |-- away_score: long (nullable = true)
 |-- away_team: struct (nullable = true)
 |    |-- away_team_gender: string (nullable = true)
 |    |-- away_team_group: string (nullable = true)
 |    |-- away_team_id: long (nullable = true)
 |    |-- away_team_name: string (nullable = true)
 |    |-- country: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- managers: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- country: struct (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- dob: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- nickname: string (nullable = true)
 |-- competition: struct (nullable = true)
 |    |-- competition_id: long (nullable = true)
 |    |-- competition_nam

In [14]:
df_jogos_ultima_temporada_champions.show()

+----------+--------------------+--------------------+-----------------+----------+--------------------+------------+--------------------+--------------------+----------+--------+------------+----------------+----------+-------------+--------------------+--------------+--------------------+
|away_score|           away_team|         competition|competition_stage|home_score|           home_team|    kick_off|        last_updated|    last_updated_360|match_date|match_id|match_status|match_status_360|match_week|     metadata|             referee|        season|             stadium|
+----------+--------------------+--------------------+-----------------+----------+--------------------+------------+--------------------+--------------------+----------+--------+------------+----------------+----------+-------------+--------------------+--------------+--------------------+
|         2|{male, null, 24, ...|{16, Champions Le...|      {26, Final}|         0|{{68, England}, m...|21:00:00.000|2022-11

In [18]:
df_final_champions = df_jogos_ultima_temporada_champions.select("home_team.home_team_name", "home_score", "away_team.away_team_name", "away_score").where("competition_stage.id = 26")

df_final_champions.show()

+-----------------+----------+--------------+----------+
|   home_team_name|home_score|away_team_name|away_score|
+-----------------+----------+--------------+----------+
|Tottenham Hotspur|         0|     Liverpool|         2|
+-----------------+----------+--------------+----------+



In [23]:
df_campeao =  df_final_champions.select(\
                                        sf.when(df_final_champions.home_score > df_final_champions.away_score, df_final_champions.home_team_name)\
                                        .otherwise(df_final_champions.away_team_name)\
                                        )\
                                .withColumnRenamed("CASE WHEN (home_score > away_score) THEN home_team_name ELSE away_team_name END", "campeao")
df_campeao.show()

+---------+
|  campeao|
+---------+
|Liverpool|
+---------+



## Qual jogador marcou o ultimo gol da edição mais recente da Champions League ?

In [26]:
final_ultima_champions_id = 22912

df_lances_da_final = spark.read.option("multiline", "true").json(f"s3://sor/football/data/events/{final_ultima_champions_id}.json")

df_lances_da_final.printSchema()


root
 |-- 50_50: struct (nullable = true)
 |    |-- outcome: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- ball_receipt: struct (nullable = true)
 |    |-- outcome: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- ball_recovery: struct (nullable = true)
 |    |-- recovery_failure: boolean (nullable = true)
 |-- block: struct (nullable = true)
 |    |-- deflection: boolean (nullable = true)
 |    |-- offensive: boolean (nullable = true)
 |-- carry: struct (nullable = true)
 |    |-- end_location: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- clearance: struct (nullable = true)
 |    |-- aerial_won: boolean (nullable = true)
 |    |-- body_part: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |-- head: boolean (nullable = true)
 |    |-- left_f

In [27]:
df_lances_da_final.show()

23/12/25 21:40:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+-----------------+-------------+-----+--------------+---------+------------+-------+--------------------+--------+--------------+--------+----------+--------------------+-----+------------+------------+------+----------+----+--------------------+------+------------------+--------------------+--------------------+----------+--------------------+--------------------+------+----+------------+--------------------+--------------------+------------+-------------------+--------------+
|50_50|     ball_receipt|ball_recovery|block|         carry|clearance|counterpress|dribble|                duel|duration|foul_committed|foul_won|goalkeeper|                  id|index|interception|    location|minute|off_camera| out|                pass|period|      play_pattern|              player|            position|possession|     possession_team|      related_events|second|shot|substitution|             tactics|                team|   timestamp|               type|under_pressure|
+-----+-------------

In [28]:
df_lances_da_final.columns

['50_50',
 'ball_receipt',
 'ball_recovery',
 'block',
 'carry',
 'clearance',
 'counterpress',
 'dribble',
 'duel',
 'duration',
 'foul_committed',
 'foul_won',
 'goalkeeper',
 'id',
 'index',
 'interception',
 'location',
 'minute',
 'off_camera',
 'out',
 'pass',
 'period',
 'play_pattern',
 'player',
 'position',
 'possession',
 'possession_team',
 'related_events',
 'second',
 'shot',
 'substitution',
 'tactics',
 'team',
 'timestamp',
 'type',
 'under_pressure']

In [31]:
# Verificando se a chave 'outcome' está como coluna no dataframe
selected_cols = [col for col in df_lances_da_final.columns if col == "outcome"]
print(selected_cols)



[]


In [33]:
# Verificando se a chave 'shot' está como coluna no dataframe
selected_cols = [col for col in df_lances_da_final.columns if col == "shot"]
print(selected_cols)

['shot']


In [44]:
lance_tipo_gol_id = 97
df_gols_da_final = df_lances_da_final.select("player.name", "minute").where(f"shot.outcome.id = {lance_tipo_gol_id}").orderBy(df_lances_da_final.minute.desc())

df_gols_da_final.show()

+------------------+------+
|              name|minute|
+------------------+------+
|Divock Okoth Origi|    86|
|     Mohamed Salah|     1|
+------------------+------+



In [46]:
df_ultimo_gol = df_gols_da_final.select("name", "minute").limit(1)

df_ultimo_gol.show()

+------------------+------+
|              name|minute|
+------------------+------+
|Divock Okoth Origi|    86|
+------------------+------+



## Quais são os estádios da base ?

In [56]:
# mapear todas as competições
df_competitions_id = df_competition.select("competition_id").distinct().orderBy("competition_id")
#df_competitions_id.show()
# mapear todas as temporadas dessas competições
df_seasons_competitions = df_competition.select("competition_id", "competition_name", "season_id", "season_name").distinct().orderBy("competition_id",sf.desc("season_id"))



# percorrer todos os jogos e guardar o nome e id dos estádios


+--------------+----------------+---------+-----------+
|competition_id|competition_name|season_id|season_name|
+--------------+----------------+---------+-----------+
|             2|  Premier League|       44|  2003/2004|
|             2|  Premier League|       27|  2015/2016|
|             7|         Ligue 1|      235|  2022/2023|
|             7|         Ligue 1|      108|  2021/2022|
|             7|         Ligue 1|       27|  2015/2016|
|             9|   1. Bundesliga|       27|  2015/2016|
|            11|         La Liga|      278|  1973/1974|
|            11|         La Liga|       90|  2020/2021|
|            11|         La Liga|       42|  2019/2020|
|            11|         La Liga|       41|  2008/2009|
|            11|         La Liga|       40|  2007/2008|
|            11|         La Liga|       39|  2006/2007|
|            11|         La Liga|       38|  2005/2006|
|            11|         La Liga|       37|  2004/2005|
|            11|         La Liga|       27|  201